# Quick DITL Example

## Introduction

This notebook demonstrates a basic Day-In-The-Life (DITL) simulation using COASTSim. We'll set up a spacecraft configuration, compute its orbit, add observation targets, run the simulation, and visualize the results.

## Step 1: Import Libraries

We start by importing the necessary libraries for configuration, ephemeris computation, visualization, and simulation.

In [ ]:
%matplotlib widget
from datetime import datetime

import numpy as np
from rust_ephem import TLEEphemeris

from conops.config.config import MissionConfig
from conops.ditl.queue_ditl import QueueDITL
from conops.visualization import plot_ditl_timeline, plot_sky_pointing

## Step 2: Create Spacecraft Configuration

Create a basic spacecraft configuration using MissionConfig. This sets up default parameters for the spacecraft bus, solar panels, payload, etc.

In [ ]:
config = MissionConfig()

## Step 3: Set up Ephemeris

Load the spacecraft's orbital ephemeris from a TLE file. This defines the
spacecraft's position over time for the simulation period. The ephemeris
pre-calculates the position of the spacecraft, along with the positions of the
Sun and Moon, for every time interval from beginning to end, with a interval of
60 seconds. We use the `rust-ephem` library to compute this for speed.

In [ ]:
ephemeris = TLEEphemeris(
    begin=datetime(2025, 12, 1, 0, 0, 0),
    end=datetime(2025, 12, 2, 0, 0, 0),
    step_size=60,
    tle="example.tle",
)

## Step 4: Create DITL and Add Observations

Create a DITL (Data in the Life) simulation object and populate it with random
astronomical observations. Each observation has a random sky position, fixed 1ks
exposure time, and a unique observation ID.

When initializing the DITL, we pass it the `MissionConfig` and `Ephemeris`
objects as arguments.

In [ ]:
ditl = QueueDITL(config=config, ephem=ephemeris)

# Add 1000 random observations to the observation queue
for i in range(1000):
    ditl.queue.add(
        ra=np.random.uniform(0, 360),
        dec=np.random.uniform(-90, 90),
        exptime=1000,
        obsid=10000 + i,
    )

## Step 5: Run the Simulation

Execute the DITL calculation to simulate the spacecraft operations, including
scheduling observations, managing power, thermal constraints, and
communications.

QueueDITL uses a queue scheduler, so it schedules targets on the fly during the
DITL. The end result is much the same as if we had pre-created a schedule.

In [ ]:
ditl.calc()

## Step 6: Visualize Results

Plot the results of the simulation, showing the sky pointing and timeline of
spacecraft operations. 

Figure 1 is an interactive map that shows the pointing of the spacecraft over
time along with various obserinv constraints that are defined by the default
configuration, most prominently the Earth (+ some limb avoidance angle), the
Moon and the Sun.

In [ ]:
_ = plot_sky_pointing(ditl, figsize=(10, 5))

In [ ]:
_ = plot_ditl_timeline(ditl, figsize=(8, 6))

In [ ]:
from conops.visualization import (
    plot_acs_mode_distribution,
    plot_data_management_telemetry,
)

plot_acs_mode_distribution(ditl)
plot_data_management_telemetry(ditl)